In [1]:
from dotenv import load_dotenv
load_dotenv("../config.env")

True

In [2]:
from coamp_graph import Graph
import coamp_graph_notebook_utils
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
coamp_graph_notebook_utils.get_all_projects()

['CCLE',
 'COLO320',
 'COLO320 DM & HSR hg38',
 'COLOtest',
 'GBM39',
 'PCAWG',
 'TCGA']

In [4]:
coamp_graph_notebook_utils.get_all_alias()

['655bda68bba7c92509522479',
 '655c060abba7c925095555da',
 'COLO320',
 'COLO320_1',
 'GBM39',
 'ccle',
 'coolest']

In [5]:
small_project_list = ['COLO320', 'GBM39']
small_df, __ = coamp_graph_notebook_utils.concat_projects(small_project_list)

In [6]:
small_graph = Graph(small_df)

Retrieved locations for 22998 GRCh38 genes in 0.08 seconds
Loaded 28555 genes from provided reference files
Gene record creation time: 1.12 seconds
31508
Matched location data for 22976 genes
Filtering features took 0.0005 seconds, resulting in 3 features
Preprocessing intervals and reformatting dataset took 0.0013 seconds
Starting CreateNodes with 3 rows
Note: 0 genes are amplified on multiple feature IDs in the same sample
TEST: 40 searches where gene's location is found
TEST: 39 searches where gene's chr on interval and gene's location is matched to an interval
TEST: 1 searches where gene's chr on interval but gene's location is NOT matched to an interval (1 unique nodes)
TEST: 0 searches where gene's chr not on interval (0 unique nodes)
Processing 40 genes took 0.0035 seconds, resulting in 26 unique nodes
0 genes were not amplified on any merged intervals
Total CreateNodes execution: 0.0035 seconds
Starting CreateEdges with 26 nodes
Building features index took 0.0000 seconds: 3 un

In [10]:
len(small_graph.Edges())

181

In [ ]:
project_list = ['655bda68bba7c92509522479', '655c060abba7c925095555da', 'ccle']
df, __ = coamp_graph_notebook_utils.concat_projects(project_list)

In [ ]:
df.shape

In [ ]:
graph = Graph(df, pdD_model='merge25kbp_cap2mbp')
nodes = graph.Nodes()
edges = graph.Edges()

In [ ]:
graph.genes_with_no_chr_match_list

In [ ]:
edges[0]

In [ ]:
baseline = Graph(df)
capped = Graph(df, pdD_maodel='gamma_capped_5m')
merge25kbp_cap2mbp = Graph(df, pdD_model='merge25kbp_cap2mbp')
merge25kbp_cap1mbp = 

In [ ]:
baseline.Nodes()[0]

In [ ]:
N = graph.total_samples
N

In [ ]:
MDM2 = [n for n in baseline.Nodes() if n['label']=='MDM2'][0]
CDK4 = [n for n in baseline.Nodes() if n['label']=='CDK4'][0]
print(len(MDM2['samples']))
len(CDK4['samples'])

In [ ]:
len(MDM2_CDK4['inter'])

In [ ]:
# MDM2-CDK4 example

MDM2_CDK4 = [edge for edge in graph.Edges() if edge['source']=='CDK4' and edge['target']=='MDM2'][0]
MDM2_CDK4['p_values']

In [ ]:
# find edges with minimum pdDs for unit testing

test_singles = [e for e in edges if e['p_d_D'] != -1 and e['p_values'][0] != -1] 
sorted_pdDs = sorted(test_singles, key = lambda x: x['p_d_D'])
[(e['source'], e['target'], e['p_d_D'], e['p_values'][0]) for e in sorted_pdDs[:10]]

In [ ]:
# find edges with minimum p_values for unit testing

test_singles = [e for e in edges if e['p_d_D'] != -1 and e['p_values'][0] != -1] 
sorted_pdDs = sorted(test_singles, key = lambda x: x['p_values'])
# [(e['source'], e['target'], e['p_d_D'], e['p_values'][0]) for e in sorted_pdDs[:10]]
[(e['source'], e['target'], e['p_values'][0]) for e in sorted_pdDs[:10]]

In [ ]:
from scipy.stats import chi2

# single interval test functions with print statements
def single_interval(edge, record_a, record_b):
		pdD = edge['p_d_D']
		geneA_samples = len(record_a['samples'])
		geneB_samples = len(record_b['samples'])
		geneAB_samples = len(edge['inter'])

		# observed
		O11 = geneAB_samples
		O12 = geneA_samples - geneAB_samples
		O21 = geneB_samples - geneAB_samples
		O22 = graph.total_samples - geneA_samples - geneB_samples + geneAB_samples
		obs = [O11, O12, O21, O22]
		print(f"observed counts: {obs}")

		# expected
		E11 = (geneA_samples + geneB_samples - geneAB_samples) * pdD
		E12 = geneA_samples * (1 - pdD)
		E21 = geneB_samples * (1 - pdD)
		E22 = graph.total_samples - (geneA_samples + geneB_samples - geneAB_samples)
		exp = [E11, E12, E21, E22] 
		print(f"expected counts: {exp}")

		return chi_squared_helper(obs, exp)

def chi_squared_helper(obs, exp):
		# apply Haldane correction if any observed category count is zero
		if 0 in obs:
			obs = [o + 1e-6 for o in obs]
			exp = [e + 1e-6 for e in exp]

		total_obs = sum(obs)
		total_exp = sum(exp)
		obs_freq = [o / total_obs for o in obs]
		exp_freq = [e / total_exp for e in exp]

		test_statistic = sum([(o-e)*(o-e)/e for o,e in zip(obs_freq, exp_freq)])
		cdf = chi2.cdf(test_statistic, df=1)
		p_val_two_sided = 1-cdf
		diagonal_residual_sum = ((obs_freq[0]-exp_freq[0]) + (obs_freq[3]-exp_freq[3]))

		odds_ratio = obs_freq[0] / exp_freq[0]
		if odds_ratio > 1e9:
			odds_ratio = 1e9 # cap ultra large to prevent infinite odds ratios

		# convert to one-sided p-value
		if diagonal_residual_sum >= 0:
			p_val_one_sided = p_val_two_sided / 2
		else:
			p_val_one_sided = 1 - (p_val_two_sided / 2)

		return p_val_one_sided, odds_ratio

In [ ]:
def test_edge(source, target):
    # node check
    genes = [node['label'] for node in nodes]
    if source not in genes:
        return f'{source} not found in graph'
    if target not in genes:
        return f'{target} not found in graph'
    source_node = [node for node in nodes if node['label'] == source][0]
    target_node = [node for node in nodes if node['label'] == target][0]
    # edge check
    st = [edge for edge in edges if edge['source']==source and edge['target']==target]
    if st:
        return single_interval(st[0], source_node, target_node)
    ts = [edge for edge in edges if edge['source']==target and edge['target']==source]
    if ts:
        return single_interval(ts[0], source_node, target_node)
    return f'edge not found in graph'

In [ ]:
test_edge('EYS','PHF3')

In [ ]:
# P-value/Q-value Distributions by Test Type

In [ ]:
p_single, p_multi_interval, p_multi_chr, p_multi_ecdna = [], [], [], []
q_single, q_multi_interval, q_multi_chr, q_multi_ecdna = [], [], [], []
na_counter = 0
for edge in edges:
    p_vals = edge.get('p_values', [-1, -1, -1, -1])
    q_vals = edge.get('q_values', [-1, -1, -1, -1])
    if p_vals == [-1, -1, -1, -1]:# and edge.get('missing_interval_data'):
        na_counter += 1
    if len(p_vals) == 4:
        p_single.append(p_vals[0])
        p_multi_interval.append(p_vals[1])
        p_multi_chr.append(p_vals[2])
        p_multi_ecdna.append(p_vals[3])
    if len(q_vals) == 4:
        q_single.append(q_vals[0])
        q_multi_interval.append(q_vals[1])
        q_multi_chr.append(q_vals[2])
        q_multi_ecdna.append(q_vals[3])
na_counter

In [ ]:
def every_fifth_percentile(data):
    data = np.array(data)
    percentiles = np.arange(5, 100, 5)
    return {p: np.percentile(data, p) for p in percentiles}

# Calculate the percentiles and format them
def print_percentiles_aligned(data):
    results = every_fifth_percentile(data)

    # Format and find max width for alignment
    formatted = {k: f"{v:.4f}" for k, v in results.items()}
    max_width = max(len(val) for val in formatted.values())

    for k in sorted(formatted):
        print(f"{k:>2}th percentile: {formatted[k]:>{max_width}}")

In [ ]:
def plot_p_values(p_values, title="p_values"):
    filtered = [p for p in p_values if p != -1]
    print(f"{len(filtered)} valid p-values from {len(p_values)} edges\n")
    
    print_percentiles_aligned(filtered)
    
    plt.figure(figsize=(8, 6))
    plt.hist(filtered, bins=50, alpha=0.7)
    plt.title(title)
    plt.xlabel("p-values")
    plt.ylabel("Frequency")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()
    


In [ ]:
plot_p_values(p_single, title=f"Single Interval P-Values (T1)")

In [ ]:
plot_p_values(q_single, title=f"Single Interval Q-Values (T1)")

In [ ]:
plot_p_values(p_multi_interval, title=f"Multi Interval P-Values (T2)")

In [ ]:
plot_p_values(q_multi_interval, title=f"Multi Interval Q-Values (T2)")

In [ ]:
# PARAMETER SWEEP FOR MERGING THRESHOLD

In [ ]:
# test merge cutoff param
project_list_small = ['COLO320', 'GBM39']
test_df, __ = coamp_graph_notebook_utils.concat_projects(project_list_small)
test_graph = Graph(test_df, merge_cutoff=50000)
print(test_graph.MERGE_CUTOFF)

In [ ]:
tmp_graph = Graph(test_df, merge_cutoff=50000, construct_graph=False)
processed_dataset = tmp_graph.preprocessed_dataset

In [ ]:
def plot_lengths_dist(lengths, ax, merge_cutoff):
    # add a subplot
    ax.hist(lengths, bins=50, alpha=0.7, color='blue')
    ax.set_title(f'merge_cutoff = {merge_cutoff}bp')
    ax.set_xlabel(f"Interval Lengths (n={len(lengths)})")
    ax.set_ylabel("Frequency")

    # plot mean line
    if lengths:
        adj_mean = sum(lengths) / len(lengths)
        ax.axvline(adj_mean, color='black', linestyle='dashed', linewidth=2, label=f'Mean = {adj_mean:.2f}')
    ax.legend()

In [ ]:
cutoff_values = list(range(10000, 51000, 5000))
num_plots = len(cutoff_values)
cols = 4
rows = (num_plots + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(4 * cols, 3 * rows))
axes = axes.flatten() 

for idx, merge_cutoff in enumerate(cutoff_values):
    ax = axes[idx]

    tmp_graph = Graph(df, merge_cutoff=merge_cutoff, construct_graph=False)
    processed_dataset = tmp_graph.preprocessed_dataset
    lengths = [interval[2] - interval[1] for list in processed_dataset['Merged_Intervals'] for interval in list if interval]

    plot_lengths_dist(lengths, ax, merge_cutoff)

# hide unused subplots if any
for j in range(len(cutoff_values), len(axes)):
    fig.delaxes(axes[j])

fig.tight_layout()
plt.show()

In [ ]:
### PLOT pdD DISTRIBUTION FOR CAPPED LENGTHS MODEL ###

In [ ]:
# df, __ = coamp_graph_notebook_utils.concat_projects(project_list)

In [ ]:
baseline = Graph(df)
capped = Graph(df, pdD_model='gamma_capped_5m')
merge25kbp_cap2mbp = Graph(df, pdD_model='merge25kbp_cap2mbp')
merge25kbp_cap1mbp = Graph(df, pdD_model='merge25kbp_cap1mbp')

In [ ]:
pdD_baseline = [edge['p_d_D'] for edge in baseline.Edges() if edge['p_d_D'] != -1]
pdD_capped = [edge['p_d_D'] for edge in capped.Edges() if edge['p_d_D'] != -1]
pdD_merge25kbp_cap2mbp = [edge['p_d_D'] for edge in merge25kbp_cap2mbp.Edges() if edge['p_d_D'] != -1]
pdD_merge25kbp_cap1mbp = [edge['p_d_D'] for edge in merge25kbp_cap1mbp.Edges() if edge['p_d_D'] != -1]

In [ ]:
pdD_baseline = pd.DataFrame(pdD_baseline, columns=['Baseline pdD Values'])
pdD_baseline.describe()

In [ ]:
pdD_capped = pd.DataFrame(pdD_capped, columns=['pdD Values'])
pdD_capped.describe()

In [ ]:
pdD_merge25kbp_cap2mbp = pd.DataFrame(pdD_merge25kbp_cap2mbp, columns=['Case 1 pdD Values'])
pdD_merge25kbp_cap2mbp.describe()

In [ ]:
pdD_merge25kbp_cap1mbp = pd.DataFrame(pdD_merge25kbp_cap1mbp, columns=['Case 2 pdD Values'])
pdD_merge25kbp_cap1mbp.describe()

In [ ]:
# overlayed
plt.figure(figsize=(8, 5))
plt.hist(pdD_baseline, bins=50, range=(0, 1), alpha=0.5, label='Baseline')
plt.hist(pdD_merge25kbp_cap1mbp, bins=50, range=(0, 1), alpha=0.5, label='Capped 1Mbp, Merged 25kbp')
plt.xlabel('pdD Value')
plt.ylabel('Frequency')
plt.title('Histogram of pdD Values')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# side by side
fig, axs = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

# histogram for baseline
axs[0].hist(pdD_baseline, bins=50, range=(0, 1), color='skyblue', edgecolor='black')
axs[0].set_title('Baseline pdD')
axs[0].set_xlabel('pdD Value')
axs[0].set_ylabel('Frequency')
axs[0].grid(True)

# histogram for capped
axs[1].hist(pdD_merge25kbp_cap2mbp, bins=50, range=(0, 1), color='salmon', edgecolor='black')
axs[1].set_title('Case 1: Merge 25kbp apart, Cap 2Mbp')
axs[1].set_xlabel('pdD Value')
axs[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
p_single = []
q_single = []
na_counter = 0
for edge in merge25kbp_cap1mbp.Edges():
    p_vals = edge.get('p_values', [-1, -1, -1, -1])
    q_vals = edge.get('q_values', [-1, -1, -1, -1])
    if p_vals == [-1, -1, -1, -1]:# and edge.get('missing_interval_data'):
        na_counter += 1
    if len(p_vals) == 4:
        p_single.append(p_vals[0])
    if len(q_vals) == 4:
        q_single.append(q_vals[0])
na_counter

In [ ]:
plot_p_values(p_single, title=f"Single Interval P-Values, merge25kbp_cap2mbp pdD")

In [ ]:
plot_p_values(q_single, title=f"Single Interval Q-Values, merge25kbp_cap2mbp pdD")